<a href="https://colab.research.google.com/github/Lenaami/SpeechSynthesisLabs/blob/main/SS_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
path = '/content/gdrive/My Drive/Colab Notebooks/Синтез речи/'

In [ ]:
book1 = 'r_hod.Result.xml'
book2 = 'tropa.Result.xml' #самый большой
book3 = 'whtguard.Result.xml'

# Парсинг xml

In [ ]:
import xml.etree.ElementTree as ET 
import numpy as np
import re

In [ ]:
def isNone(a):
    return int(a) if a is not None else -1

def isNoneWord(a):
    return a if a is not None else ''

In [ ]:
# n-граммы
def add_n_gramm(n_gramm, X):
    zero_word = np.zeros(X.shape[1], dtype = int)

    X_gram = X.copy()

    for i in range(n_gramm):
        X_gram = np.vstack((zero_word, X_gram, zero_word))

    for i in range(1, n_gramm + 1):
        X = np.hstack((X_gram[(n_gramm - i):-(n_gramm + i), :], X, X_gram[(i + n_gramm):(X_gram.shape[0] - n_gramm + i), :]))
    
    return X

In [ ]:
def get_data(file, n_gramm = 0, bySentence = False):
    X = []
    y_intonation = []
    y_pause = []
    y = []
    words = []    

    tree = ET.parse(file)
    root = tree.getroot()

    tags = ['content', 'word', 'intonation', 'pause']
    letters = ['8', '9', '10', '16', '24', '25', '26']

    for snt in root.findall('sentence'):
        features = []
        feat_wrd = []

        ys_intonation = []
        ys_pause = []
        ys = []

        words_snt = []

        phonetic_wrd = []

        count = 0
        for feat in snt:

            if feat.tag not in tags:
                continue
            
            if count != tags.index(feat.tag):
                count = 0
                ys_intonation.append(0)
                ys_pause.append(0)
                ys.append(0)

            if feat.tag == 'content':  # пунктуация и эмфазы
                feat_wrd.append(isNone(feat.get('PunktBeg')))
                feat_wrd.append(isNone(feat.get('PunktEnd')))
                feat_wrd.append(isNone(feat.get('EmphBeg')))
                feat_wrd.append(isNone(feat.get('EmphEnd')))

            if feat.tag == 'word':
                words_snt.append(isNoneWord(feat.get('original')))
                dct = feat.find('dictitem')
                if dct.get('stress_dict') is not None:
                    phonetic_wrd.append(len(words_snt) - 1) # позиция фонетического слова
                feat_wrd.append(isNone(dct.get('stress_dict'))) # ударение (номер гласной)
                feat_wrd.append(isNone(dct.get('genesys'))) # одушевленность
                feat_wrd.append(isNone(dct.get('form'))) # форма
                feat_wrd.append(isNone(dct.get('subpart_of_speech'))) # часть речи        
                feat_wrd.append(isNone(dct.get('semantics1')))
                feat_wrd.append(isNone(dct.get('semantics2')))
                lt_flags = (-1)*np.ones(len(letters), dtype=int)
                for lt in feat.findall('letter'): # флаги букв
                    if lt.get('flag') in letters:
                        lt_flags[letters.index(lt.get('flag'))] = 1       
                feat_wrd.extend(lt_flags)
                feat_wrd.append(len(re.sub(r'[^\w\s]', '', isNoneWord(feat.get('original'))))) # длина слова без символов  ## len(feat.get('original')) с учетом символов

            if feat.tag == 'intonation':
                ys_intonation.append(feat.get('type'))

            if feat.tag == 'pause':
                ys_pause.append(feat.get('type'))
                ys.append(1)

            count += 1
            
            if count == 2:
                features.append(feat_wrd)
                feat_wrd = []

            if count == 4:
                count = 0

        before = 0
        after = len(phonetic_wrd)
        for i, wrd in enumerate(words_snt):
            if i in phonetic_wrd:  # учет только фонетических слов
                before = phonetic_wrd.index(i)
                after = len(phonetic_wrd) - 1 - phonetic_wrd.index(i)
                features[i].append(before) # количество слов перед
                features[i].append(after) # количество слов после
                before += 1
            else:
                features[i].append(before)
                features[i].append(after)

        # объединение предложений в один массив
        if bySentence:
            X.append(features)
            y_intonation.append(ys_intonation)
            y_pause.append(ys_pause)
            y.append(ys)
            words.append(words_snt)
        else:
            X.extend(features)
            y_intonation.extend(ys_intonation)
            y_pause.extend(ys_pause)            
            y.extend(ys)                       
            words.extend(words_snt)

    if bySentence == False:
        X = np.asarray(X)
        X = add_n_gramm(n_gramm, X)
    
    return X, y_intonation, y_pause, y, words

In [ ]:
X_train_1, y_int_train_1, y_ps_train_1, y_train_1, words_1 = get_data(path + book2, n_gramm=2)

In [ ]:
X_train_2, y_int_train_2, y_ps_train_2, y_train_2, words_2 = get_data(path + book3, n_gramm=2)

In [ ]:
# Объединение корпусов (2 и 3)

X_train = np.vstack((X_train_1, X_train_2))

y_train = []
y_train.extend(y_train_1)
y_train.extend(y_train_2)

y_ps_train = []
y_ps_train.extend(y_ps_train_1)
y_ps_train.extend(y_ps_train_2)

y_int_train = []
y_int_train.extend(y_int_train_1)
y_int_train.extend(y_int_train_2)

In [ ]:
# Тестовый корпус (1)

X_test, y_int_test, y_ps_test, y_test, words_test = get_data(path + book1, n_gramm=2)

#Классификация пауз

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def fit_clf(clf, X_train, y_train):
    clf.fit(X_train, y_train)
    return clf

def predict_clf(clf, X_test, y_test, metric='binary'):
    y_pred = clf.predict(X_test)

    print(metrics.classification_report(y_test, y_pred))

    print('F1-score (%s):\t%0.3f' % (metric, metrics.f1_score(y_test, y_pred, average=metric)))
    print('Recall (%s): \t%0.3f' % (metric, metrics.recall_score(y_test, y_pred, average=metric)))
    print('Precision (%s):\t%0.3f' % (metric,metrics.precision_score(y_test, y_pred, average=metric)))
    print('Accuracy:\t\t%0.3f' % metrics.accuracy_score(y_test, y_pred))

    print('\nConfusion matrix:\n', pd.DataFrame(metrics.confusion_matrix(y_test, y_pred)))

    return y_pred

def cross_val_clf(clf, X, y, metric='f1'):
    scores = cross_val_score(estimator=clf, X=X, y=y, cv=5, scoring=metric)
    df_scores = pd.DataFrame({'Srocc Val Scores (F1)': scores})
    print(df_scores)
    print('Average:\t%0.3f\n' % np.mean(scores))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Decision Tree

In [ ]:
tree = DecisionTreeClassifier()
    
cross_val_clf(tree, X_train, y_train)

tree = fit_clf(tree, X_train, y_train)
y_pred = predict_clf(tree, X_test, y_test)

   Srocc Val Scores (F1)
0               0.952575
1               0.955050
2               0.959241
3               0.938533
4               0.946732
Average:	0.950

              precision    recall  f1-score   support

           0       0.99      0.98      0.98       970
           1       0.94      0.95      0.95       298

    accuracy                           0.97      1268
   macro avg       0.96      0.97      0.97      1268
weighted avg       0.97      0.97      0.97      1268

F1-score (binary):	0.947
Recall (binary): 	0.953
Precision (binary):	0.940
Accuracy:		0.975

Confusion matrix:
      0    1
0  952   18
1   14  284


## Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=15)

cross_val_clf(rf, X_train, y_train)

rf = fit_clf(rf, X_train, y_train)
y_pred = predict_clf(rf, X_test, y_test)

   Srocc Val Scores (F1)
0               0.963832
1               0.965414
2               0.971905
3               0.950552
4               0.962200
Average:	0.963

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       970
           1       0.99      0.94      0.97       298

    accuracy                           0.98      1268
   macro avg       0.99      0.97      0.98      1268
weighted avg       0.98      0.98      0.98      1268

F1-score (binary):	0.966
Recall (binary): 	0.943
Precision (binary):	0.989
Accuracy:		0.984

Confusion matrix:
      0    1
0  967    3
1   17  281


## Naive Baies



In [ ]:
nb = GaussianNB(var_smoothing=1e-5) 

cross_val_clf(nb, X_train, y_train)

nb = fit_clf(nb, X_train, y_train)
y_pred = predict_clf(nb, X_test, y_test)

   Srocc Val Scores (F1)
0               0.727149
1               0.724684
2               0.734301
3               0.708370
4               0.733291
Average:	0.726

              precision    recall  f1-score   support

           0       0.88      0.96      0.92       970
           1       0.81      0.59      0.68       298

    accuracy                           0.87      1268
   macro avg       0.85      0.77      0.80      1268
weighted avg       0.87      0.87      0.86      1268

F1-score (binary):	0.682
Recall (binary): 	0.591
Precision (binary):	0.807
Accuracy:		0.871

Confusion matrix:
      0    1
0  928   42
1  122  176


## Logisric Regression

In [ ]:
lr = LogisticRegression()

cross_val_clf(lr, X_train, y_train)

lr = fit_clf(lr, X_train, y_train)
y_pred = predict_clf(lr, X_test, y_test)

   Srocc Val Scores (F1)
0               0.936746
1               0.934878
2               0.946012
3               0.913275
4               0.925129
Average:	0.931

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       970
           1       0.97      0.89      0.93       298

    accuracy                           0.97      1268
   macro avg       0.97      0.94      0.95      1268
weighted avg       0.97      0.97      0.97      1268

F1-score (binary):	0.928
Recall (binary): 	0.886
Precision (binary):	0.974
Accuracy:		0.968

Confusion matrix:
      0    1
0  963    7
1   34  264


# Классификация по типам пауз и интонационных контуров

In [ ]:
# Типы пауз в используемых корпусах

print(set(y_ps_train))
print(set(y_ps_test))

{0, 'long', 'x-long', 'spelling', 'weak'}
{0, 'long', 'x-long', 'spelling', 'weak'}


In [ ]:
# Типы интонационных контуров в используемых корпусах

print(set(y_int_train))
print(set(y_int_test))

{0, '40', '50', '30', '70', '110', '11'}
{0, '40', '50', '30', '70', '110', '11'}


## Учитывая нулевой класс

Для каждого классификатора:
1. Классификация по типу паузы
2. Классификация по типу интонационного контура

### Decision Tree

In [ ]:
tree_2 = DecisionTreeClassifier()
    
cross_val_clf(tree_2, X_train, y_ps_train, metric='f1_macro')

tree_2 = fit_clf(tree_2, X_train, y_ps_train)
y_pred = predict_clf(tree_2, X_test, y_ps_test, metric='macro')

   Srocc Val Scores (F1)
0               0.867442
1               0.771254
2               0.759452
3               0.750449
4               0.701569
Average:	0.770

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       970
        long       0.98      1.00      0.99        91
    spelling       1.00      1.00      1.00         1
        weak       0.91      0.93      0.92       204
      x-long       0.00      0.00      0.00         2

    accuracy                           0.97      1268
   macro avg       0.78      0.78      0.78      1268
weighted avg       0.97      0.97      0.97      1268

F1-score (macro):	0.778
Recall (macro): 	0.782
Precision (macro):	0.775
Accuracy:		0.972

Confusion matrix:
      0   1  2    3  4
0  952   0  0   18  0
1    0  91  0    0  0
2    0   0  1    0  0
3   15   0  0  189  0
4    0   2  0    0  0


In [ ]:
tree_3 = DecisionTreeClassifier()
    
cross_val_clf(tree_3, X_train, y_int_train, metric='f1_macro')

tree_3 = fit_clf(tree_3, X_train, y_int_train)
y_pred = predict_clf(tree_3, X_test, y_int_test, metric='macro')

   Srocc Val Scores (F1)
0               0.717130
1               0.685636
2               0.718948
3               0.698500
4               0.701718
Average:	0.704

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       970
          11       0.93      0.97      0.95        87
         110       0.89      0.90      0.89       183
          30       0.67      0.50      0.57        12
          40       0.83      1.00      0.91        10
          50       1.00      0.67      0.80         3
          70       0.33      0.33      0.33         3

    accuracy                           0.96      1268
   macro avg       0.81      0.76      0.78      1268
weighted avg       0.96      0.96      0.96      1268

F1-score (macro):	0.777
Recall (macro): 	0.764
Precision (macro):	0.805
Accuracy:		0.961

Confusion matrix:
      0   1    2  3   4  5  6
0  951   0   18  0   0  0  1
1    1  84    0  1   1  0  0
2   13   3  165  1   0  0  1
3    1   3  

### Random Forest

In [ ]:
rf_2 = RandomForestClassifier(n_estimators=15) 

cross_val_clf(rf_2, X_train, y_ps_train, metric='f1_macro')

rf_2 = fit_clf(rf_2, X_train, y_ps_train)
y_pred = predict_clf(rf_2, X_test, y_ps_test, metric='macro')

   Srocc Val Scores (F1)
0               0.680351
1               0.747765
2               0.759069
3               0.779176
4               0.617269
Average:	0.717

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       970
        long       0.98      1.00      0.99        91
    spelling       1.00      1.00      1.00         1
        weak       0.97      0.92      0.94       204
      x-long       0.00      0.00      0.00         2

    accuracy                           0.98      1268
   macro avg       0.79      0.78      0.78      1268
weighted avg       0.98      0.98      0.98      1268

F1-score (macro):	0.784
Recall (macro): 	0.782
Precision (macro):	0.787
Accuracy:		0.981

Confusion matrix:
      0   1  2    3  4
0  965   0  0    5  0
1    0  91  0    0  0
2    0   0  1    0  0
3   17   0  0  187  0
4    0   2  0    0  0


In [ ]:
rf_3 = RandomForestClassifier(n_estimators=15)

cross_val_clf(rf_3, X_train, y_int_train, metric='f1_macro')

rf_3 = fit_clf(rf_3, X_train, y_int_train)
y_pred = predict_clf(rf_3, X_test, y_int_test, metric='macro')

   Srocc Val Scores (F1)
0               0.570505
1               0.543288
2               0.524875
3               0.539155
4               0.559745
Average:	0.548

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       970
          11       0.78      0.84      0.81        87
         110       0.90      0.89      0.89       183
          30       0.33      0.08      0.13        12
          40       0.00      0.00      0.00        10
          50       0.00      0.00      0.00         3
          70       0.00      0.00      0.00         3

    accuracy                           0.95      1268
   macro avg       0.43      0.40      0.40      1268
weighted avg       0.93      0.95      0.94      1268

F1-score (macro):	0.403
Recall (macro): 	0.401
Precision (macro):	0.427
Accuracy:		0.948

Confusion matrix:
      0   1    2  3  4  5  6
0  966   0    4  0  0  0  0
1    4  73   10  0  0  0  0
2   18   2  162  1  0  0  0
3    0   8    3 

### Naive Baies

In [ ]:
nb_2 = GaussianNB(var_smoothing=1e-5)

cross_val_clf(nb_2, X_train, y_ps_train, metric='f1_macro')

nb_2 = fit_clf(nb_2, X_train, y_ps_train)
y_pred = predict_clf(nb_2, X_test, y_ps_test, metric='macro')

   Srocc Val Scores (F1)
0               0.551282
1               0.689748
2               0.685619
3               0.614275
4               0.495626
Average:	0.607

              precision    recall  f1-score   support

           0       0.95      0.89      0.92       970
        long       0.99      0.87      0.92        91
    spelling       1.00      1.00      1.00         1
        weak       0.59      0.76      0.67       204
      x-long       0.07      0.50      0.12         2

    accuracy                           0.87      1268
   macro avg       0.72      0.80      0.73      1268
weighted avg       0.89      0.87      0.87      1268

F1-score (macro):	0.726
Recall (macro): 	0.803
Precision (macro):	0.719
Accuracy:		0.865

Confusion matrix:
      0   1  2    3   4
0  861   0  0  107   2
1    2  79  0    0  10
2    0   0  1    0   0
3   48   0  0  155   1
4    0   1  0    0   1


In [ ]:
nb_3 = GaussianNB(var_smoothing=1e-5)

cross_val_clf(nb_3, X_train, y_int_train, metric='f1_macro')

nb_3 = fit_clf(nb_3, X_train, y_int_train)
y_pred = predict_clf(nb_3, X_test, y_int_test, metric='macro')

   Srocc Val Scores (F1)
0               0.446821
1               0.444499
2               0.467644
3               0.445916
4               0.435079
Average:	0.448

              precision    recall  f1-score   support

           0       0.96      0.75      0.85       970
          11       0.75      0.75      0.75        87
         110       0.40      0.85      0.54       183
          30       0.14      0.17      0.15        12
          40       0.00      0.00      0.00        10
          50       0.00      0.00      0.00         3
          70       0.00      0.00      0.00         3

    accuracy                           0.75      1268
   macro avg       0.32      0.36      0.33      1268
weighted avg       0.85      0.75      0.78      1268

F1-score (macro):	0.327
Recall (macro): 	0.359
Precision (macro):	0.322
Accuracy:		0.752

Confusion matrix:
      0   1    2  3  4  5  6
0  732   8  219  5  0  0  6
1    7  65    9  2  0  1  3
2   19   5  155  1  0  0  3
3    1   5    3 

### Logistic Regression

In [ ]:
lr_2 = LogisticRegression() 

cross_val_clf(lr_2, X_train, y_ps_train, metric='f1_macro')

lr_2 = fit_clf(lr_2, X_train, y_ps_train)
y_pred = predict_clf(lr_2, X_test, y_ps_test, metric='macro')

   Srocc Val Scores (F1)
0               0.568748
1               0.712253
2               0.716886
3               0.696573
4               0.558448
Average:	0.651

              precision    recall  f1-score   support

           0       0.96      0.99      0.98       970
        long       0.97      0.98      0.97        91
    spelling       0.00      0.00      0.00         1
        weak       0.96      0.82      0.89       204
      x-long       0.00      0.00      0.00         2

    accuracy                           0.96      1268
   macro avg       0.58      0.56      0.57      1268
weighted avg       0.96      0.96      0.96      1268

F1-score (macro):	0.567
Recall (macro): 	0.559
Precision (macro):	0.578
Accuracy:		0.962

Confusion matrix:
      0   1  2    3  4
0  963   0  0    7  0
1    0  89  0    0  2
2    0   1  0    0  0
3   36   0  0  168  0
4    0   2  0    0  0


In [ ]:
lr_3 = LogisticRegression() 

cross_val_clf(lr_3, X_train, y_int_train, metric='f1_macro')

lr_3 = fit_clf(lr_3, X_train, y_int_train)
y_pred = predict_clf(lr_3, X_test, y_int_test, metric='macro')

   Srocc Val Scores (F1)
0               0.382431
1               0.378983
2               0.391218
3               0.385577
4               0.383793
Average:	0.384

              precision    recall  f1-score   support

           0       0.96      0.99      0.98       970
          11       0.72      0.78      0.75        87
         110       0.86      0.79      0.82       183
          30       0.00      0.00      0.00        12
          40       0.00      0.00      0.00        10
          50       0.00      0.00      0.00         3
          70       0.00      0.00      0.00         3

    accuracy                           0.93      1268
   macro avg       0.36      0.37      0.36      1268
weighted avg       0.91      0.93      0.92      1268

F1-score (macro):	0.364
Recall (macro): 	0.366
Precision (macro):	0.363
Accuracy:		0.927

Confusion matrix:
      0   1    2  3  4  5  6
0  963   0    7  0  0  0  0
1    5  68   12  1  1  0  0
2   34   4  144  1  0  0  0
3    0   8    4 

## Учитывая только ненулевые классы

In [ ]:
def get_data_klass(X, y, y_ps, y_int):
    new_X = []
    new_y_ps = []
    new_y_int = []
    for i in range(len(y)):
        if y[i] == 1:
            new_X.append(X[i])
            new_y_ps.append(y_ps[i])
            new_y_int.append(y_int[i])            
    
    new_X = np.asarray(new_X)
    return new_X, new_y_ps, new_y_int

In [ ]:
X_train_klass, y_ps_train_klass, y_int_train_klass = get_data_klass(X_train, y_train, y_ps_train, y_int_train)

In [ ]:
X_test_klass, y_ps_test_klass, y_int_test_klass = get_data_klass(X_test, y_test,y_ps_test, y_int_test)

### Decision Tree

In [ ]:
tree_4 = DecisionTreeClassifier() 

cross_val_clf(tree_4, X_train_klass, y_ps_train_klass, metric='f1_macro')

tree_4 = fit_clf(tree_4, X_train_klass, y_ps_train_klass)
y_pred = predict_clf(tree_4, X_test_klass, y_ps_test_klass, metric='macro')

   Srocc Val Scores (F1)
0               0.791165
1               0.698472
2               0.526820
3               0.783776
4               0.695893
Average:	0.699

              precision    recall  f1-score   support

        long       0.98      1.00      0.99        91
    spelling       1.00      1.00      1.00         1
        weak       1.00      1.00      1.00       204
      x-long       0.00      0.00      0.00         2

    accuracy                           0.99       298
   macro avg       0.74      0.75      0.75       298
weighted avg       0.99      0.99      0.99       298

F1-score (macro):	0.747
Recall (macro): 	0.750
Precision (macro):	0.745
Accuracy:		0.993

Confusion matrix:
     0  1    2  3
0  91  0    0  0
1   0  1    0  0
2   0  0  204  0
3   2  0    0  0


In [ ]:
tree_5 = DecisionTreeClassifier() 

cross_val_clf(tree_5, X_train_klass, y_int_train_klass, metric='f1_macro')

tree_5 = fit_clf(tree_5, X_train_klass, y_int_train_klass)
y_pred = predict_clf(tree_5, X_test_klass, y_int_test_klass, metric='macro')

   Srocc Val Scores (F1)
0               0.692425
1               0.650763
2               0.696817
3               0.679019
4               0.669869
Average:	0.678

              precision    recall  f1-score   support

          11       0.97      0.95      0.96        87
         110       0.97      0.98      0.98       183
          30       0.64      0.58      0.61        12
          40       0.83      1.00      0.91        10
          50       1.00      0.67      0.80         3
          70       0.33      0.33      0.33         3

    accuracy                           0.95       298
   macro avg       0.79      0.75      0.76       298
weighted avg       0.95      0.95      0.95       298

F1-score (macro):	0.764
Recall (macro): 	0.753
Precision (macro):	0.790
Accuracy:		0.946

Confusion matrix:
     0    1  2   3  4  5
0  83    1  2   1  0  0
1   1  179  1   0  0  2
2   2    3  7   0  0  0
3   0    0  0  10  0  0
4   0    0  0   1  2  0
5   0    1  1   0  0  1


### Random Forest

In [ ]:
rf_4 = RandomForestClassifier(n_estimators=15) 

cross_val_clf(rf_4, X_train_klass, y_ps_train_klass, metric='f1_macro')

rf_4 = fit_clf(rf_4, X_train_klass, y_ps_train_klass)
y_pred = predict_clf(rf_4, X_test_klass, y_ps_test_klass, metric='macro')

   Srocc Val Scores (F1)
0               0.861885
1               0.698941
2               0.524659
3               0.540726
4               0.687454
Average:	0.663

              precision    recall  f1-score   support

        long       0.98      1.00      0.99        91
    spelling       1.00      1.00      1.00         1
        weak       1.00      1.00      1.00       204
      x-long       0.00      0.00      0.00         2

    accuracy                           0.99       298
   macro avg       0.74      0.75      0.75       298
weighted avg       0.99      0.99      0.99       298

F1-score (macro):	0.747
Recall (macro): 	0.750
Precision (macro):	0.745
Accuracy:		0.993

Confusion matrix:
     0  1    2  3
0  91  0    0  0
1   0  1    0  0
2   0  0  204  0
3   2  0    0  0


In [ ]:
rf_5 = RandomForestClassifier(n_estimators=15)

cross_val_clf(rf_5, X_train_klass, y_int_train_klass, metric='f1_macro')

rf_5 = fit_clf(rf_5, X_train_klass, y_int_train_klass)
y_pred = predict_clf(rf_5, X_test_klass, y_int_test_klass, metric='macro')

   Srocc Val Scores (F1)
0               0.514429
1               0.513018
2               0.562370
3               0.536692
4               0.514112
Average:	0.528

              precision    recall  f1-score   support

          11       0.81      0.86      0.83        87
         110       0.93      0.99      0.96       183
          30       1.00      0.08      0.15        12
          40       0.88      0.70      0.78        10
          50       0.00      0.00      0.00         3
          70       0.00      0.00      0.00         3

    accuracy                           0.89       298
   macro avg       0.60      0.44      0.45       298
weighted avg       0.88      0.89      0.86       298

F1-score (macro):	0.454
Recall (macro): 	0.439
Precision (macro):	0.602
Accuracy:		0.886

Confusion matrix:
     0    1  2  3  4  5
0  75   10  0  1  1  0
1   2  181  0  0  0  0
2   8    3  1  0  0  0
3   3    0  0  7  0  0
4   3    0  0  0  0  0
5   2    1  0  0  0  0


### Naive Baies

In [ ]:
nb_4 = GaussianNB(var_smoothing=1e-5)

cross_val_clf(nb_4, X_train_klass, y_ps_train_klass, metric='f1_macro')

nb_4 = fit_clf(nb_4, X_train_klass, y_ps_train_klass)
y_pred = predict_clf(nb_4, X_test_klass, y_ps_test_klass, metric='macro')

   Srocc Val Scores (F1)
0               0.726739
1               0.726196
2               0.534057
3               0.479576
4               0.649860
Average:	0.623

              precision    recall  f1-score   support

        long       0.99      0.89      0.94        91
    spelling       1.00      1.00      1.00         1
        weak       1.00      1.00      1.00       204
      x-long       0.08      0.50      0.14         2

    accuracy                           0.96       298
   macro avg       0.77      0.85      0.77       298
weighted avg       0.99      0.96      0.97       298

F1-score (macro):	0.769
Recall (macro): 	0.846
Precision (macro):	0.768
Accuracy:		0.960

Confusion matrix:
     0  1    2   3
0  81  0    0  10
1   0  1    0   0
2   0  0  203   1
3   1  0    0   1


In [ ]:
nb_5 = GaussianNB(var_smoothing=1e-5)

cross_val_clf(nb_5, X_train_klass, y_int_train_klass, metric='f1_macro')

nb_5 = fit_clf(nb_5, X_train_klass, y_int_train_klass)
y_pred = predict_clf(nb_5, X_test_klass, y_int_test_klass, metric='macro')

   Srocc Val Scores (F1)
0               0.456889
1               0.451720
2               0.474249
3               0.459170
4               0.449166
Average:	0.458

              precision    recall  f1-score   support

          11       0.83      0.78      0.80        87
         110       0.92      0.94      0.93       183
          30       0.17      0.17      0.17        12
          40       0.00      0.00      0.00        10
          50       0.00      0.00      0.00         3
          70       0.00      0.00      0.00         3

    accuracy                           0.81       298
   macro avg       0.32      0.31      0.32       298
weighted avg       0.81      0.81      0.81       298

F1-score (macro):	0.317
Recall (macro): 	0.315
Precision (macro):	0.319
Accuracy:		0.812

Confusion matrix:
     0    1  2  3  4  5
0  68   10  3  0  1  5
1   5  172  3  0  0  3
2   5    4  2  0  1  0
3   2    0  3  0  5  0
4   1    0  1  1  0  0
5   1    1  0  0  1  0


### Logistic Regression (useless)

In [ ]:
lr_4 = LogisticRegression() 

cross_val_clf(lr_4, X_train_klass, y_ps_train_klass, metric='f1_macro')

lr_4 = fit_clf(lr_4, X_train_klass, y_ps_train_klass)
y_pred = predict_clf(lr_4, X_test_klass, y_ps_test_klass, metric='macro')

   Srocc Val Scores (F1)
0               0.663879
1               0.676067
2               0.507265
3               0.525606
4               0.693388
Average:	0.613

              precision    recall  f1-score   support

        long       0.97      1.00      0.98        91
    spelling       0.00      0.00      0.00         1
        weak       1.00      1.00      1.00       204
      x-long       0.00      0.00      0.00         2

    accuracy                           0.99       298
   macro avg       0.49      0.50      0.50       298
weighted avg       0.98      0.99      0.98       298

F1-score (macro):	0.496
Recall (macro): 	0.500
Precision (macro):	0.492
Accuracy:		0.990

Confusion matrix:
     0  1    2  3
0  91  0    0  0
1   1  0    0  0
2   0  0  204  0
3   2  0    0  0


In [ ]:
lr_5 = LogisticRegression() 

cross_val_clf(lr_5, X_train_klass, y_int_train_klass, metric='f1_macro')

lr_5 = fit_clf(lr_5, X_train_klass, y_int_train_klass)
y_pred = predict_clf(lr_5, X_test_klass, y_int_test_klass, metric='macro')

   Srocc Val Scores (F1)
0               0.329306
1               0.345180
2               0.350029
3               0.363210
4               0.369945
Average:	0.352

              precision    recall  f1-score   support

          11       0.71      0.80      0.75        87
         110       0.89      0.97      0.93       183
          30       0.00      0.00      0.00        12
          40       0.00      0.00      0.00        10
          50       0.00      0.00      0.00         3
          70       0.00      0.00      0.00         3

    accuracy                           0.83       298
   macro avg       0.27      0.30      0.28       298
weighted avg       0.76      0.83      0.79       298

F1-score (macro):	0.280
Recall (macro): 	0.295
Precision (macro):	0.267
Accuracy:		0.829

Confusion matrix:
     0    1  2  3  4  5
0  70   16  0  0  0  1
1   6  177  0  0  0  0
2   8    4  0  0  0  0
3  10    0  0  0  0  0
4   3    0  0  0  0  0
5   2    1  0  0  0  0


# Комбинирование классификаторов

Вариант, со специально обученными классификаторами

1. Random Forest (определение места паузы)
2. Naive Baies (определение типа паузы)
3. Decision Tree (определение интонационного контура)

In [ ]:
def clf_prosodic(X, clf, clf_ps, clf_int):
    y_pred = clf.predict(X) # определение пауз

    new_X = []
    for i in range(len(y_pred)):
        if y_pred[i] == 1:
            new_X.append(X[i])

    y_ps = clf_ps.predict(new_X)
    y_int = clf_int.predict(new_X)
    
    y_ps_pred = []
    y_int_pred = []

    count = 0
    for i in range(len(y_pred)):
        if y_pred[i] == 1:
            y_ps_pred.append(y_ps[count])
            y_int_pred.append(y_int[count])
            count += 1
        else:
            y_ps_pred.append(0)
            y_int_pred.append(0)

    return y_pred, y_ps_pred, y_int_pred

In [ ]:
def clf_score(y_pred, y, metric='binary'):

    print(metrics.classification_report(y, y_pred))

    print('F1-score (%s):\t%0.3f' % (metric, metrics.f1_score(y, y_pred, average=metric)))
    print('Recall (%s): \t%0.3f' % (metric, metrics.recall_score(y, y_pred, average=metric)))
    print('Precision (%s):\t%0.3f' % (metric,metrics.precision_score(y, y_pred, average=metric)))
    print('Accuracy:\t\t%0.3f' % metrics.accuracy_score(y, y_pred))

    print('\nConfusion matrix:\n', pd.DataFrame(metrics.confusion_matrix(y, y_pred)))

In [ ]:
y_pred, y_ps_pred, y_int_pred = clf_prosodic(X_test, rf, nb_4, tree_5)

Результаты:
1. Определение места паузы
2. Определение типа паузы
3. Определение интонационного контура

In [ ]:
clf_score(y_pred, y_test, metric='macro')

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       970
           1       0.99      0.94      0.97       298

    accuracy                           0.98      1268
   macro avg       0.99      0.97      0.98      1268
weighted avg       0.98      0.98      0.98      1268

F1-score (macro):	0.978
Recall (macro): 	0.970
Precision (macro):	0.986
Accuracy:		0.984

Confusion matrix:
      0    1
0  967    3
1   17  281


In [ ]:
clf_score(y_ps_pred, y_ps_test, metric='macro')

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       970
        long       0.99      0.89      0.94        91
    spelling       1.00      1.00      1.00         1
        weak       0.98      0.91      0.95       204
      x-long       0.08      0.50      0.14         2

    accuracy                           0.97      1268
   macro avg       0.81      0.86      0.80      1268
weighted avg       0.98      0.97      0.98      1268

F1-score (macro):	0.803
Recall (macro): 	0.860
Precision (macro):	0.808
Accuracy:		0.975

Confusion matrix:
      0   1  2    3   4
0  967   0  0    3   0
1    0  81  0    0  10
2    0   0  1    0   0
3   17   0  0  186   1
4    0   1  0    0   1


In [ ]:
clf_score(y_int_pred, y_int_test, metric='macro')

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       970
          11       0.96      0.93      0.95        87
         110       0.96      0.90      0.93       183
          30       0.64      0.58      0.61        12
          40       0.83      1.00      0.91        10
          50       1.00      0.67      0.80         3
          70       0.33      0.33      0.33         3

    accuracy                           0.97      1268
   macro avg       0.82      0.77      0.79      1268
weighted avg       0.97      0.97      0.97      1268

F1-score (macro):	0.788
Recall (macro): 	0.773
Precision (macro):	0.816
Accuracy:		0.972

Confusion matrix:
      0   1    2  3   4  5  6
0  967   0    3  0   0  0  0
1    3  81    0  2   1  0  0
2   14   1  165  1   0  0  2
3    0   2    3  7   0  0  0
4    0   0    0  0  10  0  0
5    0   0    0  0   1  2  0
6    0   0    1  1   0  0  1
